## Import necessary modules

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### Import the data

In [2]:
bom = pd.read_csv("data/bom.movie_gross.csv.gz")
imdb_names = pd.read_csv("data/imdb.name.basics.csv.gz")
# imdb_title_akas = pd.read_csv("data/imdb.title.akas.csv.gz")
imdb_title_basics = pd.read_csv("data/imdb.title.basics.csv.gz")
# imdb_title_princ = pd.read_csv("data/imdb.title.principals.csv.gz")
imdb_title_ratings = pd.read_csv("data/imdb.title.ratings.csv.gz")
tmdb = pd.read_csv("data/tmdb.movies.csv.gz")
rt_movie_info = pd.read_csv('data/rt.movie_info.tsv.gz', sep='\t')
rt_reviews = pd.read_csv("data/rt.reviews.tsv.gz", sep='\t', encoding='latin-1')
tn_movie_budgets = pd.read_csv("data/tn.movie_budgets.csv.gz", sep=',')

In [41]:
inner_bom_tmdb = bom.merge(tmdb, left_on='title', right_on='title')

In [45]:
imdb_title_basics = pd.read_csv("data/imdb.title.basics.csv.gz")

In [23]:
brad_pitt = imdb_names[imdb_names['primary_name'] == 'Brad Pitt'].copy()

In [24]:
brad_pitt

,nconst,primary_name,birth_year,death_year,primary_profession,known_for_titles
24026,nm0000093,Brad Pitt,1963.0,NaN,"actor,producer,soundtrack","tt1210166,tt0114746,tt0356910,tt0816711"


In [56]:
brad_pitt[['movie_1', 'movie_2', 'movie_3', 'movie_4']] = brad_pitt.apply(lambda x: pd.Series(x['known_for_titles'].split(',')), axis=1)

In [55]:
brad_pitt.apply(lambda x: pd.Series(x['known_for_titles'].split(',')), axis=1).stack()

24026  0    tt1210166
       1    tt0114746
       2    tt0356910
       3    tt0816711
dtype: object

In [60]:
brad_pitt_melt = brad_pitt.melt(id_vars='primary_name', value_vars=['movie_1', 'movie_2', 'movie_3', 'movie_4'])

In [61]:
brad_pitt_melt.merge(brad_pitt, left_on='primary_name', right_on='primary_name')

,primary_name,variable,value,nconst,birth_year,death_year,primary_profession,known_for_titles,kft,movie_1,movie_2,movie_3,movie_4
0,Brad Pitt,movie_1,tt1210166,nm0000093,1963.0,NaN,"actor,producer,soundtrack","tt1210166,tt0114746,tt0356910,tt0816711",tt1210166,tt1210166,tt0114746,tt0356910,tt0816711
1,Brad Pitt,movie_2,tt0114746,nm0000093,1963.0,NaN,"actor,producer,soundtrack","tt1210166,tt0114746,tt0356910,tt0816711",tt1210166,tt1210166,tt0114746,tt0356910,tt0816711
2,Brad Pitt,movie_3,tt0356910,nm0000093,1963.0,NaN,"actor,producer,soundtrack","tt1210166,tt0114746,tt0356910,tt0816711",tt1210166,tt1210166,tt0114746,tt0356910,tt0816711
3,Brad Pitt,movie_4,tt0816711,nm0000093,1963.0,NaN,"actor,producer,soundtrack","tt1210166,tt0114746,tt0356910,tt0816711",tt1210166,tt1210166,tt0114746,tt0356910,tt0816711


In [62]:
brad_pitt_melt

,primary_name,variable,value
0,Brad Pitt,movie_1,tt1210166
1,Brad Pitt,movie_2,tt0114746
2,Brad Pitt,movie_3,tt0356910
3,Brad Pitt,movie_4,tt0816711


In [97]:
imdb_head = imdb_title_basics.head().copy()

In [98]:
imdb_head['genres_list'] = imdb_head.genres.str.split(',')

In [121]:
s = imdb_head.apply(lambda x: pd.Series(x['genres_list']), axis=1).stack().reset_index(level=1, drop=True)

In [125]:
s.name = 'Genres'
imdb_genres = imdb_head.join(s).copy()

In [133]:
genres_dummies = pd.get_dummies(imdb_genres.Genres).sum(level=0)

In [136]:
imdb_head.join(genres_dummies).drop(['genres', 'genres_list'], axis=1)

,tconst,primary_title,original_title,start_year,runtime_minutes,Action,Biography,Comedy,Crime,Drama,Fantasy
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,1,0,0,1,1,0
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,0,1,0,0,1,0
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,0,0,0,0,1,0
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,0,0,1,0,1,0
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,0,0,1,0,1,1


### One-Hot Encode Genres for Every Movie

In [140]:
def OHE_genres(movie_list):
    movie_list['genres_list'] = movie_list.genres.str.split(',')
    s = movie_list.apply(lambda x: pd.Series(x['genres_list']), 
                                             axis=1).stack().reset_index(level=1, drop=True)
    s.name = 'Genres'
    imdb_genres = movie_list.join(s).copy()
    genres_dummies = pd.get_dummies(imdb_genres.Genres).sum(level=0)
    ml = movie_list.join(genres_dummies).drop(['genres', 'genres_list'], axis=1)
    return ml

In [77]:
inner_bom_imdb = bom.merge(imdb_title_basics, left_on='title', right_on='primary_title')

In [81]:
inner_bom_imdb.set_index('tconst', inplace=True)

In [82]:
inner_bom_imdb.head()

,title,studio,domestic_gross,foreign_gross,year,primary_title,original_title,start_year,runtime_minutes,genres
tconst,,,,,,,,,,
tt0435761,Toy Story 3,BV,415000000.0,652000000,2010,Toy Story 3,Toy Story 3,2010,103.0,"Adventure,Animation,Comedy"
tt1375666,Inception,WB,292600000.0,535700000,2010,Inception,Inception,2010,148.0,"Action,Adventure,Sci-Fi"
tt0892791,Shrek Forever After,P/DW,238700000.0,513900000,2010,Shrek Forever After,Shrek Forever After,2010,93.0,"Adventure,Animation,Comedy"
tt1325004,The Twilight Saga: Eclipse,Sum.,300500000.0,398000000,2010,The Twilight Saga: Eclipse,The Twilight Saga: Eclipse,2010,124.0,"Adventure,Drama,Fantasy"
tt1228705,Iron Man 2,Par.,312400000.0,311500000,2010,Iron Man 2,Iron Man 2,2010,124.0,"Action,Adventure,Sci-Fi"


In [142]:
inner_bom_imdb = OHE_genres(inner_bom_imdb)

In [143]:
inner_bom_imdb.columns

Index(['title', 'studio', 'domestic_gross', 'foreign_gross', 'year',
       'primary_title', 'original_title', 'start_year', 'runtime_minutes',
       'Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Family', 'Fantasy', 'History', 'Horror',
       'Music', 'Musical', 'Mystery', 'News', 'Romance', 'Sci-Fi', 'Sport',
       'Thriller', 'War', 'Western'],
      dtype='object')